# Wavelet Transform and Run-Length Encoding
## Compression
#### compress

In [ ]:
def wavelet_compress(sampling_rate, num_bits_run_len, max_prd, thresh_perc_approx,
                        thresh_perc_d5, thresh_perc_d4_d1, data):
    import pywt
    from pywt import wavedec
    import numpy as np
    import pandas as pd
    from scipy.signal import detrend
    import copy 
    import json
    import os
    
    FS = sampling_rate
    COEFF_LENGTHS = {'cA5': 100, 'cD5': 100, 'cD4': 100, 'cD3': 100, 'cD2': 100, 'cD1': 100}
    NUM_BITS_RUN_LEN = num_bits_run_len
    MAX_PRD = max_prd
    THRESH_PERC_APPROX = thresh_perc_approx
    THRESH_PERC_D5 = thresh_perc_d5
    THRESH_PERC_D4_D1 = thresh_perc_d4_d1
    NUM_SAMPLES = len(data)
    
    coeffs = wavelet_decomposition(data)
    for key in coeffs.keys():
        COEFF_LENGTHS[key] = len(coeffs[key])
    
    coeffs_thresholded, binary_map, coeffs_orig = threshold_energy(coeffs)
    
    coeffs_scaled, scaling_factors = scale_coeffs(coeffs_thresholded)
    
    num_bits, PRD = calculate_num_bits(data, coeffs_scaled, binary_map, scaling_factors)
    coeffs_quantized = do_quantization(coeffs_scaled, num_bits)
    
    if num_bits < 9:
        coeffs_quantized_combined = combine_coefficients(coeffs_quantized, binary_map)
    binary_map_combined = combine_coefficients(binary_map)
    
    coeffs_quantized_compressed, num_bits_last_byte_coeffs = compress_coefficients(coeffs_quantized_combined, num_bits)
    binary_map_initial_state, binary_map_compressed, num_bits_last_byte_binary_map = compress_binary_map(binary_map_combined)
    
    os.mkdir('compressed_ppg')
    if num_bits < 9:
        np.savetxt('compressed_ppg/ppg_cqc.txt',coeffs_quantized_compressed,fmt='%3d') 
        with open('compressed_ppg/ppg_nblbc.txt','w') as f:
            f.write('%d' % num_bits_last_byte_coeffs)
    else:
        np.savetxt('compressed_ppg/ppg_cqc.txt',coeffs_quantized_combined,fmt='%3d') 

    with open('compressed_ppg/ppg_bmis.txt','w') as f:
        f.write('%d' % binary_map_initial_state)
    np.savetxt('compressed_ppg/ppg_bmc.txt',binary_map_compressed,fmt='%3d') 
    with open('compressed_ppg/ppg_nblbbm.txt','w') as f:
        f.write('%d' % num_bits_last_byte_binary_map)
    with open('compressed_ppg/ppg_nb.txt','w') as f:
        f.write('%d' % num_bits)
    json.dump(scaling_factors, open("compressed_ppg/ppg_sf.txt",'w'))
    json.dump(COEFF_LENGTHS, open("compressed_ppg/ppg_cl.txt",'w'))

## Decompression

#### read compressed files

In [29]:
coeffs_quantized_compressed = np.loadtxt('compressed_eda/eda_cqc.txt').astype('int32')

with open('compressed_eda/eda_nblbc.txt','r') as f:
    num_bits_last_byte_coeffs = int(f.read())

with open('compressed_eda/eda_bmis.txt','r') as f:
    binary_map_initial_state = int(f.read())

binary_map_compressed = np.loadtxt('compressed_eda/eda_bmc.txt').astype('int32')

with open('compressed_eda/eda_nblbbm.txt','r') as f:
    num_bits_last_byte_binary_map = int(f.read())

with open('compressed_eda/eda_nb.txt','r') as f:
    num_bits = int(f.read())

scaling_factors = json.load(open('compressed_eda/eda_sf.txt'))

COEFF_LENGTHS = json.load(open("compressed_eda/eda_cl.txt"))

#### decompress

In [ ]:
def wavelet_decompress(coeffs_quantized_compressed, num_bits_last_byte_coeffs, binary_map_initial_state,
                       binary_map_compressed, num_bits_last_byte_binary_map, num_bits,
                       scaling_factors, COEFF_LENGTHS):
    
    binary_map_decompressed = decompress_binary_map(binary_map_compressed, binary_map_initial_state, num_bits_last_byte_binary_map)
    coeffs_decompressed = decompress_coefficients(coeffs_quantized_compressed, num_bits, num_bits_last_byte_coeffs)
    coeffs_reconstructed = remap_coeffs(coeffs_decompressed, binary_map_decompressed)
    coeffs_unscaled = unscale_coeffs(coeffs_reconstructed, scaling_factors, num_bits)
    data_reconstructed = wavelet_reconstruction(coeffs_unscaled)
    
    return data_reconstructed

______________________
## Sub-functions Defined 

### wavelet decomposition
*The wavelet used in this algorithm is bior4.4, with 5 levels of decomposition.*

In [3]:
def wavelet_decomposition(sig):
    cA5, cD5, cD4, cD3, cD2, cD1 = wavedec(sig, 'bior4.4', level=5)
    coeffs = {'cA5': cA5, 'cD5': cD5, 'cD4': cD4, 'cD3': cD3, 'cD2': cD2, 'cD1': cD1}

    return coeffs

### wavelet reconstruction
*the wavelet is bior4.4 with 5 levels of decomposition*

In [4]:
def wavelet_reconstruction(coeffs):
    reconstructed = pywt.waverec([coeffs['cA5'], coeffs['cD5'], coeffs['cD4'], coeffs['cD3'], 
                                    coeffs['cD2'], coeffs['cD1']], 'bior4.4')
    return reconstructed

### threshold wavelet coefficients to a percentage of total energy
*Different levels of decomposition are thresholded at different energy percentages.*  
1. calculate the energy of all the coefficients
1. compute threshold for each coefficient matrix
1. keep corresponding coefficients that are above the threshold

In [5]:
def threshold_energy(coeffs):
    #make a deep copy of coeffs to retain the original version
    coeffs_orig = copy.deepcopy(coeffs)

    binary_map = {}
    nonzero_coeff_count = {}

    for key in coeffs.keys():
        #sort the absolute value of the coefficients in descending order
        tmp_coeffs = np.sort(np.abs(coeffs[key]))[::-1]

        #calculate the threshold for retaining some percentage of the energy
        if key == 'cA5':
            thresh_perc = THRESH_PERC_APPROX
        elif key == 'cD5':
            thresh_perc = THRESH_PERC_D5
        else:
            thresh_perc = THRESH_PERC_D4_D1

        energy_thresholded = thresh_perc*energy(tmp_coeffs)
        energy_tmp = 0
        for coeff in tmp_coeffs:
            energy_tmp = energy_tmp + coeff**2

            if energy_tmp >= energy_thresholded:
                threshold = coeff
                break

        #set any coefficients below the threshold to zero
        tmp_coeffs = coeffs[key]
        inds_to_zero = np.where((tmp_coeffs < threshold) & (tmp_coeffs > -threshold))[0]
        tmp_coeffs[inds_to_zero] = 0

        #create the binary map
        binary_map_tmp = np.ones(len(coeffs[key])).astype(int)
        binary_map_tmp[inds_to_zero] = 0

        #update the various dictionaries
        coeffs[key] = tmp_coeffs
        binary_map[key] = binary_map_tmp
        nonzero_coeff_count[key] = len(tmp_coeffs)
        
    return coeffs, binary_map, coeffs_orig

### scale the wavelet coefficients to the [0,1] range
*two scaling factors: a shift factor and a multiplication factor*

In [6]:
def scale_coeffs(coeffs):
    coeffs_scaled = {}
    scaling_factors = {}

    for key in coeffs.keys():
        shift_factor = np.min(coeffs[key])
        coeffs_tmp = coeffs[key]-shift_factor

        scale_factor = np.max(coeffs_tmp)
        coeffs_tmp = coeffs_tmp/scale_factor

        scaling_factors[key] = {'shift_factor': shift_factor, 'scale_factor': scale_factor}
        coeffs_scaled[key] = coeffs_tmp

    return coeffs_scaled, scaling_factors

### unscale the coefficients back to their original scaling

In [7]:
def unscale_coeffs(coeffs_reconstructed, scaling_factors, bits, do_plot=False):
    coeffs_unscaled = {}

    for key in coeffs_reconstructed.keys():
        tmp_coeffs_unscaled = coeffs_reconstructed[key]/(2**bits)
        tmp_coeffs_unscaled = tmp_coeffs_unscaled*scaling_factors[key]['scale_factor']
        tmp_coeffs_unscaled = tmp_coeffs_unscaled + scaling_factors[key]['shift_factor']

        #now replace the NaN values with 0
        nan_inds = np.where(np.isnan(tmp_coeffs_unscaled))[0]
        tmp_coeffs_unscaled[nan_inds] = 0

        coeffs_unscaled[key] = tmp_coeffs_unscaled

    return coeffs_unscaled

### calculate the lowest possible number of bits to quantize the wavelet coefficients such that the PRD is above the threshold
1. quantize the signal starting at 8 bits
1. unquantize and reconstruct the signal
1. calculate the PRD. Repeat with 1 fewer bit (ie, 7 bits)
1. repeat

In [8]:
def calculate_num_bits(orig_sig, coeffs_scaled, binary_map, scaling_factors):
    #starting at 12 bits, keep decreasing the number of bits in the quantization
    #until the PRD is above some threshold
    num_bits = 13

    #initialize PRD to 0 so the while loop can run
    PRD = 0

    #keep track of PRD per number of bits
    PRD_dict = {}

    while (num_bits >= 5) and (PRD <= MAX_PRD):
        #decrement the number of bits
        num_bits = num_bits-1

        coeffs_quantized = do_quantization(coeffs_scaled, num_bits)

        #rescale the coefficients
        coeffs_unscaled = unscale_coeffs(None, coeffs_quantized, scaling_factors, num_bits)

        #do the inverse dwt
        data_reconstructed = wavelet_reconstruction(coeffs_unscaled, None, None)
        
        #calculate PRD
        PRD = calculate_PRD(orig_sig, data_reconstructed)
        PRD_dict[num_bits] = PRD

    #if we went over the PRD, go back up by one bit
    if PRD > MAX_PRD:
        num_bits = num_bits+1
        PRD = PRD_dict[num_bits]

    return num_bits, PRD

### combine all the wavelet coefficients into one continuous array
*done for each decomposition level and for binary map*

In [9]:
def combine_coefficients(coeffs, binary_map=None):
    coeffs_combined = []

    #loop through each of the wavelet decompositions
    #(or coefficient matrices) and remove all zero values
    #based on the binary map
    if binary_map is not None:
        for key in coeffs.keys():
            inds_to_keep = np.where(binary_map[key]==1)[0]
            coeffs[key] = coeffs[key][inds_to_keep]

    #add each array to coeffs_combined
    coeffs_combined.extend(coeffs['cA5'])
    coeffs_combined.extend(coeffs['cD5'])
    coeffs_combined.extend(coeffs['cD4'])
    coeffs_combined.extend(coeffs['cD3'])
    coeffs_combined.extend(coeffs['cD2'])
    coeffs_combined.extend(coeffs['cD1'])

    return coeffs_combined

### map the wavelet coefficients (and the binary map) back to their original decomposition levels
*necessary prerequisite for reconstruction of the time domain waveform*

In [10]:
def remap_coeffs(coeffs, binary_map):
    coeffs_remapped = np.zeros(len(binary_map))*np.nan
    inds_to_set = np.where(np.asarray(binary_map)==1)[0]
    coeffs_remapped[inds_to_set] = coeffs

    wavelet_remapped = {}
    counter = 0
    wavelet_remapped['cA5'] = coeffs_remapped[counter:counter+COEFF_LENGTHS['cA5']]

    counter = counter + COEFF_LENGTHS['cA5']
    wavelet_remapped['cD5'] = coeffs_remapped[counter:counter+COEFF_LENGTHS['cD5']]

    counter = counter + COEFF_LENGTHS['cD5']
    wavelet_remapped['cD4'] = coeffs_remapped[counter:counter+COEFF_LENGTHS['cD4']]

    counter = counter + COEFF_LENGTHS['cD4']
    wavelet_remapped['cD3'] = coeffs_remapped[counter:counter+COEFF_LENGTHS['cD3']]

    counter = counter + COEFF_LENGTHS['cD3']
    wavelet_remapped['cD2'] = coeffs_remapped[counter:counter+COEFF_LENGTHS['cD2']]

    counter = counter + COEFF_LENGTHS['cD2']
    wavelet_remapped['cD1'] = coeffs_remapped[counter:counter+COEFF_LENGTHS['cD1']]

    return wavelet_remapped

### quantization
*input: the selected largest wavelet coefficients*  

In [11]:
def do_quantization(coeffs, bits):
    quantized_coeffs = {}

    for key in coeffs.keys():
        sig = coeffs[key]
        sig = sig*(2**bits-1)
        sig = np.round(sig)
        sig = np.array(sig).astype(int)

        quantized_coeffs[key] = sig
        
    return quantized_coeffs

### compress the wavelet coefficients
*combine bits into bytes*

In [12]:
def compress_coefficients(coeffs, num_bits):

    binary_string = ''

    for coeff in coeffs:
        #convert each coefficient value to binary in num_bits number of bits
        binary_string = binary_string + format(coeff, '0%ib' % num_bits)

    #loop through sets of 8 bits in the binary string and convert to a byte
    byte_array = []
    for i in range(int(len(binary_string)/8)):
        byte_tmp = binary_string[i*8:(i+1)*8]
        byte_tmp = int(byte_tmp, 2)
        byte_array.append(byte_tmp)

    #check if there are any remaining bits that don't divide evenly into 8
    #note the number of bits in this last byte for conversion back to int
    #later on
    num_bits_last_byte = 8
    if len(binary_string)%8 != 0:
        byte_tmp = binary_string[(i+1)*8:(i+1)*8 + len(binary_string)%8]
        num_bits_last_byte = len(byte_tmp)
        byte_tmp = int(byte_tmp, 2)
        byte_array.append(byte_tmp)

    return byte_array, num_bits_last_byte

### decompress the previously compressed wavelet coefficients

In [13]:
def decompress_coefficients(coeffs_compressed, num_bits, num_bits_last_byte):

    binary_string = ''

    #convert each coefficient value to binary in 8 number of bits
    #note that the very last value in the the binary map may not be
    #a full 8 bits. so convert that based on num_bits_last_byte
    coeffs_len = len(coeffs_compressed)
    for i in range(coeffs_len):
        if i == coeffs_len-1:
            binary_string = binary_string + format(coeffs_compressed[i], '0%ib' % num_bits_last_byte)
        else:
            binary_string = binary_string + format(coeffs_compressed[i], '08b')


    #loop through sets of num_bits bits in the binary string and convert to a byte
    byte_array = []
    for i in range(int(len(binary_string)/num_bits)):
        byte_tmp = binary_string[i*num_bits:(i+1)*num_bits]
        byte_tmp = int(byte_tmp, 2)
        byte_array.append(byte_tmp)


    return byte_array

### compress the binary map using variable length run-length encoding (RLE)

In [14]:
def compress_binary_map(binary_map):
    #define a state machine that loops through each entry in the binary map and 
    #creates the compressed representation. 

    #the last run count won't be included in the compressed representation, so 
    #just append one more value at the end of the binary map to trigger the last 
    #compression value. make a local deep copy so that the original is not affected
    binary_map = copy.deepcopy(binary_map)
    binary_map.append(int(not binary_map[-1]))


    CURRENT_STATE = binary_map[0]
    run_count = 0
    binary_string = ''

    #loop through each value in the binary map
    for val in binary_map:

        #if the current binary map value is the same as the previous one, just increment the run count
        if val == CURRENT_STATE:
            run_count = run_count + 1

        #otherwise, encode the current run count 
        else:

            #handle cases where run count <= 3
            if run_count == 1:
                binary_string_tmp = '00'

            elif run_count == 2:
                binary_string_tmp = '01'

            elif run_count == 3:
                binary_string_tmp = '10'

            #otherwise, if the run count > 3
            else:
                #calculate the number bits required to represent the run count
                num_bits_run_count = len(format(run_count, 'b'))

                #build a binary string
                binary_string_tmp = ''

                #first bit represents that the run count > 3
                binary_string_tmp = binary_string_tmp + '11'

                #next 4 bits represent the number of bits that will define the run count
                binary_string_tmp = binary_string_tmp + format(num_bits_run_count, '0%ib' % NUM_BITS_RUN_LEN)

                #next number of bits is variable, and is the actual run count
                #may be up to 15 bits assuming NUM_BITS_RUN_LEN=4
                binary_string_tmp = binary_string_tmp + format(run_count, 'b')

            #print(str(run_count) + ', ' + binary_string_tmp)
            #pdb.set_trace()

            #append the binary string
            binary_string = binary_string + binary_string_tmp

            #reset the run count 
            run_count = 1

        #update the current state
        CURRENT_STATE = val


    #convert the binary string into a buffer of 8 bit bytes 
    byte_array = []
    for i in range(int(len(binary_string)/8)):
        byte_tmp = binary_string[i*8:(i+1)*8]
        byte_tmp = int(byte_tmp, 2)
        byte_array.append(byte_tmp)


    #check if there are any remaining bits that don't divide evenly into 8
    num_bits_last_byte = 8
    if len(binary_string)%8 != 0:
        byte_tmp = binary_string[(i+1)*8:(i+1)*8 + len(binary_string)%8]
        num_bits_last_byte = len(byte_tmp)
        byte_tmp = int(byte_tmp, 2)
        byte_array.append(byte_tmp)


    #return the initial state (ie, the first value in binary map), and the RLE binary map
    return binary_map[0], byte_array, num_bits_last_byte

### decompress the previously compressed binary map

In [15]:
def decompress_binary_map(binary_map_compressed, binary_map_initial_state, num_bits_last_byte):

    #first convert 8 bit numbers into a binary string
    binary_string = ''

    #convert each coefficient value to binary in 8 number of bits
    #note that the very last value in the the binary map may not be
    #a full 8 bits. so convert that based on num_bits_last_byte
    binary_map_len = len(binary_map_compressed)
    for i in range(binary_map_len):
        if i == binary_map_len-1:
            binary_string = binary_string + format(binary_map_compressed[i], '0%ib' % num_bits_last_byte)
        else:
            binary_string = binary_string + format(binary_map_compressed[i], '08b')


    #define a state machine that loops through each entry in the binary map and 
    #creates the uncompressed representation. 
    READ_HEADER = 0
    READ_NUM_BITS = 1
    READ_RUN_LEN = 2
    state = READ_HEADER

    run_type = binary_map_initial_state
    header = ''
    binary_array = np.array([])


    #loop through each value in the binary map
    for val in binary_string:

        #read the header
        if state == READ_HEADER:
            header = header + val

            if len(header) == 2:
                #run count 1
                if header == '00':
                    binary_array = np.concatenate((binary_array, np.ones(1)*run_type))
                    run_type = int(not run_type)
                    state = READ_HEADER

                #run count 2
                if header == '01':
                    binary_array = np.concatenate((binary_array, np.ones(2)*run_type))
                    run_type = int(not run_type)
                    state = READ_HEADER

                #run count 3
                if header == '10':
                    binary_array = np.concatenate((binary_array, np.ones(3)*run_type))
                    run_type = int(not run_type)
                    state = READ_HEADER

                #run count > 3
                if header == '11':
                    state = READ_NUM_BITS
                    num_bits = ''


                #reset header 
                header = ''

            continue

        #read number of bits
        if state == READ_NUM_BITS:


            num_bits = num_bits + val

            if len(num_bits) == 4:
                num_bits_run_len = int(num_bits, 2)
                run_len = ''

                state = READ_RUN_LEN

            continue


        #read run length
        if state == READ_RUN_LEN:
            run_len = run_len + val

            if len(run_len) == num_bits_run_len:
                run_len = int(run_len, 2)
                binary_array = np.concatenate((binary_array, np.ones(run_len)*run_type))
                run_type = int(not run_type)
                state = READ_HEADER

            continue


    return binary_array

### calculate signal energy

In [16]:
def energy(sig):
    return np.sum(sig**2)